In [1]:
import torch
from torch import nn,save,load
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard

2024-02-23 01:58:52.663400: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# def Datasets(images_dir):
#     ids = os.walk(images_dir)

#     for root,_,files in ids:
#         images = files

#     return files

In [3]:
class Discriminator(nn.Module):
    def __init__(self,img_dim):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(img_dim,128),
            nn.LeakyReLU(0.01),
            nn.Linear(128,1),
            nn.Sigmoid(),
        )
        
    def forward(self,x):
        return self.disc(x)

In [4]:
class Generator(nn.Module):
    def __init__(self,latent_space,img_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(latent_space,256),
            nn.LeakyReLU(0.01),
            nn.Linear(256,img_dim),
            nn.Tanh(),
        )
        
    def forward(self,x):
        return self.gen(x)

In [5]:
# device ="cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4
latent_space = 64
img_dim = 28*28*1 #784
batch_size = 32
num_epochs = 50

In [10]:
disc = Discriminator(img_dim)
gen = Generator(latent_space,img_dim)

fixed_noise = torch.randn((batch_size,latent_space))
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
# dataset = Datasets("sketch_data/")
# dataset = transforms(dataset)

dataset = datasets.ImageFolder(root="/home/zok/Music/Models/GAN/Simple_GAN/image/", transform=transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f"logs/fake")
writer_real = SummaryWriter(f"logs/real")
step = 0

FileNotFoundError: Couldn't find any class folder in /home/zok/Music/Models/GAN/Simple_GAN/image/.

In [ ]:
for epoch in range(num_epochs):
    for batch_idx, (img, _) in enumerate(loader):
        img = img.view(-1, 784)
        batch_size = img.shape[0]

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
        noise = torch.randn(batch_size, latent_space)
        f_img = gen(noise)
        r_img = disc(img).view(-1)
        lossD_img = criterion(r_img,torch.ones_like(r_img))
        f_img = disc(f_img).view(-1)
        lossD_fimg = criterion(f_img,torch.zeros_like(r_img))
        lossD = (lossD_img+lossD_fimg)/2
        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()

        ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
        # where the second option of maximizing doesn't suffer from
        # saturating gradients
        noise = torch.randn(batch_size, latent_space)
        f_img = gen(noise)
        lossGf = disc(f_img).view(-1)
        lossG = criterion(lossGf, torch.ones_like(lossGf))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()
        
        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
                      Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = img.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
                step += 1

In [18]:
# with open('gan_disc.pt', 'wb') as f: 
#     save(disc.state_dict(), f) 

In [ ]:
# with open('gan_gen.pt', 'wb') as f: 
#     save(gen.state_dict(), f) 

In [8]:
with open('gan_disc.pt', 'rb') as f: 
    disc.load_state_dict(load(f))

In [14]:
with open('gan_gen.pt', 'rb') as f: 
    gen.load_state_dict(load(f))

In [15]:
noise = torch.randn(latent_space)
fa = gen(noise).reshape(28,28)
transform = transforms.ToPILImage()
img = transform(fa)
img.show()

In [14]:
img

In [15]:
img_tensor = transforms.ToTensor()(img).unsqueeze(0)
img_tensor = img_tensor.view(-1)
rl = disc(img_tensor)
rl

tensor([0.4886], grad_fn=<SigmoidBackward0>)